In [930]:
from scipy.io import loadmat
from scipy.stats import zscore
from decimal import *
import pandas as pd
import numpy as np
import datetime as dt
import os
import sys
import math
import pandas as pd
import numpy as np
from pprint import pprint
import json
import requests
import datetime
from datetime import date
import os
import xmltodict
from pandas.io.json import json_normalize

In [931]:
initial_value = "Anne_v3_"

#v1: no colors, initial run to make sure the code works and generates the rough glyph design
#v2: colors added, topologies fixed
#v3: refinement, added facet

In [932]:
with open('oura_2020-08-19_AG.json') as json_data:
    dataJ = json.load(json_data)
    
sleep = json_normalize(dataJ, 'sleep')
dfs = pd.DataFrame(sleep)

readiness = json_normalize(dataJ, 'readiness')
dfr = pd.DataFrame(readiness)

activity = json_normalize(dataJ, 'activity')
dfa = pd.DataFrame(activity)

In [933]:
df = dfa.merge(dfr, on='summary_date')

In [934]:
df = df.merge(dfs, on = 'summary_date')
df.rename( columns={'Unnamed: 0':'index'}, inplace=True )

In [935]:
df['bed_start'] = pd.to_datetime(df['bedtime_start'])
df['sum_date'] = pd.to_datetime(df['summary_date'])

In [936]:
df['date'] = df['sum_date'].dt.date

In [937]:
df = df.set_index(pd.DatetimeIndex(df['date']))

In [938]:
df1 = df.loc['2020-07-14':'2020-07-27']

In [939]:
df1 = df1.reset_index(drop=True)

In [940]:
df1.replace({0:0.001}, inplace = True)
df1.fillna(0.001, inplace=True)

In [941]:
def make_interpolater(left_min, left_max, right_min, right_max): 
    # Figure out how 'wide' each range is  
    leftSpan = left_max - left_min  
    rightSpan = right_max - right_min  

    # Compute the scale factor between left and right values 
    scaleFactor = float(rightSpan) / float(leftSpan) 

    # create interpolation function using pre-calculated scaleFactor
    def interp_fn(value):
        return right_min + (value-left_min)*scaleFactor

    return interp_fn

In [942]:
def Reverse(lst): 
    lst.reverse() 
    return lst 

In [943]:
for column in df1[['score_x', 'score_meet_daily_targets', 'score_move_every_hour', 'score_recovery_time', 'score_stay_active', 'score_training_frequency', 'score_training_volume', 'non_wear', 'high', 'inactive', 'rest',  'low', 'medium', 'cal_total', 'daily_movement', 'score_y', 'score_activity_balance', 'score_hrv_balance', 'score_previous_day', 'score_previous_night', 'score_recovery_index', 'score_resting_hr', 'score_sleep_balance', 'score_temperature', 'awake', 'breath_average', 'deep', 'light', 'rem', 'hr_average', 'hr_lowest', 'score', 'score_alignment', 'score_deep', 'score_disturbances', 'score_efficiency', 'score_latency', 'score_rem', 'score_total']]:
    columnSeriesObj = df1[column]
    col_min = df1[column].min() 
    col_max = df1[column].max() 
    col_list = df1[column].tolist()
    scalar = make_interpolater(col_min + .0002, col_max, .03, .43) 
    col_interp = [scalar(x) for x in col_list] 
    df1[(str(column) + str('_scale'))] = col_interp

In [944]:
xlist = list(range(-300, 295, 85))

In [945]:
xlist = [xlist]*2
xaxis = [y for x in xlist for y in x]

In [946]:
df1['xroot'] = xaxis
df1['yroot'] = [55, 55, 55, 55, 55, 55, 55, -65, -65, -65, -65, -65, -65, -65]
df1['zroot'] = 3

In [947]:
df1['bed_start'] = df1['bed_start'].dt.tz_localize(tz='UTC')

In [948]:
df1['bed_start'] = df1['bed_start'].dt.tz_convert(tz='America/Los_Angeles')

In [949]:
df1['bed_start'] = df1['bed_start'].dt.time

In [950]:
llist = []
for index, row in df1.iterrows():
    if (row['bed_start'].hour > 12):
        # random day + 1
        x = '2020-01-01' + ' ' + str(row['bed_start'])
        llist.append(x)
    elif (row['bed_start'].hour < 12):
        # random day
        #rr = pd.to_datetime['2020-01-01' + ' ' + str(row['BB'])]
        x = x = '2020-01-02' + ' ' + str(row['bed_start'])
        llist.append(x)

In [951]:
df1['converted_date'] = pd.to_datetime(llist)

In [952]:
# 'converted_date' = bedtime start as timestamp
df1['converted_date'] = (df1['converted_date'].astype(int) / 10**9)

In [953]:
col_list = df1['converted_date'].values.tolist()
scalar = make_interpolater(1577851200, 1577937600, 105, -120) 
col_interp = [scalar(x) for x in col_list] 

In [954]:
df1['bedtime_start'] = col_interp

# Food DataFrame

In [955]:
dffd = pd.read_csv("servings.csv")

In [956]:
dffd = dffd.set_index(pd.to_datetime(dffd['Day']))


In [957]:
dffd = dffd.loc['2020-07-14':'2020-8-04']


In [958]:
dffd = dffd.reset_index(drop=True)

In [959]:
dffd['Time'] = pd.to_datetime(dffd['Time'])
dffd['mealtime'] = [d.time() for d in dffd['Time']]

In [960]:
df3 = dffd.groupby(['Day', 'mealtime']).sum()

In [961]:
df3 = df3[['energy', 'protein', 'fat', 'carbs', 'fiber', 'sugar']]

In [962]:
tmlist = []
for index, row in df3.iterrows():
    totalmeal = row['protein'] + row['carbs'] + row['fat']
    tmlist.append(totalmeal)

In [963]:
df3['totalmeal'] = tmlist
#df3['totalmeal'] = df3['totalmeal'].fillna(0)

In [964]:
df3['p'] = df3['protein']/ df3['totalmeal']
df3['ft'] = df3['fat']/ df3['totalmeal']
df3['c'] = df3['carbs']/ df3['totalmeal']
df3['s'] = df3['sugar']/ df3['carbs']
df3['fb'] = df3['fiber']/ df3['carbs']

In [965]:
for column in df3[['p', 'ft', 'c']]:
    columnSeriesObj = df3[column]
    col_min = df3[column].min() 
    col_max = df3[column].max() 
    col_list = df3[column].tolist()
    scalar = make_interpolater(col_min + .0002, col_max, 1, 6.5) 
    col_interp = [scalar(x) for x in col_list] 
    df3[(str(column) + str('_scale'))] = col_interp

In [966]:
for column in df3[['s', 'fb']]:
    columnSeriesObj = df3[column]
    col_min = df3[column].min() 
    col_max = df3[column].max() 
    col_list = df3[column].tolist()
    scalar = make_interpolater(col_min + .0002, col_max, .3, 1.5) 
    col_interp = [scalar(x) for x in col_list] 
    df3[(str(column) + str('_scale'))] = col_interp

In [967]:
# cal_chunk is every  100 calories per meal
df3['cal_chunk'] = df3['energy']/100
df3['cal_chunk'] = df3['cal_chunk'].round(0).astype(int)

In [968]:
df3 = df3.reset_index()

In [969]:
df3['date'] = pd.to_datetime(df3['Day'])

In [970]:
ylist = []
for index, row in df3.iterrows():
    x = '2020-01-01' + ' ' + str(row['mealtime'])
    ylist.append(x)

In [971]:
df3['mtime'] = pd.to_datetime(ylist)

In [972]:
df3['mtimestamp'] = (df3['mtime'].astype(int) / 10**9)

In [973]:
col_list = df3['mtimestamp'].values.tolist()
scalar = make_interpolater(1577851200, 1577937600, 105, -120) 
col_interp = [scalar(x) for x in col_list] 

In [974]:
df3['y3'] = col_interp

In [975]:
#df3.head()

## df4 blood glucose

In [976]:
dfblood = pd.read_csv("blood_glucose.csv")

In [977]:
dfg = dfblood.loc[dfblood['type'] == 'glucose']
dfg = dfg[['value', 'date', 'time', 'gki']]
col_list = ['bg', 'date', 'time', 'gki']
dfg.columns = col_list
#dfg.head()

In [978]:
dfk = dfblood.loc[dfblood['type'] == 'ketone']
dfk = dfk[['value', 'date', 'time']]
col_list = ['k', 'date', 'time']
dfk.columns = col_list
#dfk.head()

In [979]:
col_min = dfg['bg'].min() 
col_max = dfg['bg'].max() 
col_list = dfg['bg'].tolist()
scalar = make_interpolater(col_min + .0002, col_max, .03, .58) 
col_interp = [scalar(x) for x in col_list] 
dfg['bg_scale'] = col_interp

In [980]:
col_min = dfg['gki'].min() 
col_max = dfg['gki'].max() 
col_list = dfg['gki'].tolist()
scalar = make_interpolater(col_min + .0002, col_max, .03, .58) 
col_interp = [scalar(x) for x in col_list] 
dfg['gki_scale'] = col_interp

In [981]:
col_min = dfk['k'].min() 
col_max = dfk['k'].max() 
col_list = dfk['k'].tolist()
scalar = make_interpolater(col_min + .0002, col_max, .03, .58) 
col_interp = [scalar(x) for x in col_list] 
dfk['k_scale'] = col_interp

In [982]:
dfg['time'] = pd.to_datetime(dfg['time'])

In [983]:
dfk = dfk.fillna(0)
dfk['time'] = pd.to_datetime(dfk['time'])

In [984]:
#dfsurvey.loc[dfsurvey["Are_you_in_isolation"] == 'Yes', 'Are_you_in_isolation'] = 1
dfk.loc[(dfk['time'] > '07:00:00') & (dfk['time'] < '09:00:00'), 'time_num'] = 8
dfk.loc[(dfk['time'] > '09:00:00') & (dfk['time'] < '11:00:00'), 'time_num'] = 10
dfk.loc[(dfk['time'] > '11:00:00') & (dfk['time'] < '13:00:00'), 'time_num'] = 12
dfk.loc[(dfk['time'] > '13:00:00') & (dfk['time'] < '15:00:00'), 'time_num'] = 14
dfk.loc[(dfk['time'] > '15:00:00') & (dfk['time'] < '17:00:00'), 'time_num'] = 16
dfk.loc[(dfk['time'] > '17:00:00') & (dfk['time'] < '19:00:00'), 'time_num'] = 18

In [985]:
dfk['time_num'] = dfk['time_num'].astype(int)
dfk['t_num'] = pd.to_datetime(dfk['time_num'], format = '%H')

In [986]:
dfk['t_num_time'] = dfk['t_num'].dt.time

In [987]:
klist = []
for index, row in dfk.iterrows():
    x = '2020-01-01' + ' ' + str(row['t_num_time'])
    klist.append(x)

In [988]:
dfk['tmt'] = pd.to_datetime(klist)

In [989]:
dfk['tmtstamp'] = (dfk['tmt'].astype(int) / 10**9)

In [990]:
col_list = dfk['tmtstamp'].values.tolist()
scalar = make_interpolater(1577851200, 1577937600, 105, -120) 
col_interp = [scalar(x) for x in col_list] 

In [991]:
dfk['y4'] = col_interp

In [992]:
dfg.loc[(dfg['time'] > '07:00:00') & (dfg['time'] < '09:00:00'), 'time_num'] = 8
dfg.loc[(dfg['time'] > '09:00:00') & (dfg['time'] < '11:00:00'), 'time_num'] = 10
dfg.loc[(dfg['time'] > '11:00:00') & (dfg['time'] < '13:00:00'), 'time_num'] = 12
dfg.loc[(dfg['time'] > '13:00:00') & (dfg['time'] < '15:00:00'), 'time_num'] = 14
dfg.loc[(dfg['time'] > '15:00:00') & (dfg['time'] < '17:00:00'), 'time_num'] = 16
dfg.loc[(dfg['time'] > '17:00:00') & (dfg['time'] < '19:00:00'), 'time_num'] = 18

In [993]:
dfg = dfg.fillna(0)

In [994]:
dfg['time_num'] = dfg['time_num'].astype(int)
dfg['t_num'] = pd.to_datetime(dfg['time_num'], format = '%H')

In [995]:
dfg['t_num_time'] = dfg['t_num'].dt.time

In [996]:
glist = []
for index, row in dfg.iterrows():
    x = '2020-01-01' + ' ' + str(row['t_num_time'])
    glist.append(x)

In [997]:
dfg['tmt'] = pd.to_datetime(glist)

In [998]:
dfg['tmtstamp'] = (dfg['tmt'].astype(int) / 10**9)

In [999]:
col_list = dfg['tmtstamp'].values.tolist()
scalar = make_interpolater(1577851200, 1577937600, 105, -120) 
col_interp = [scalar(x) for x in col_list] 

In [1000]:
dfg['y4'] = col_interp

In [1001]:
df4 = dfg.merge(dfk,on = ['date', 'time_num'], how = 'outer')

In [1002]:
df4['date'] = pd.to_datetime(df4['date'])

In [1003]:
df4 = df4.loc[(df4['date'] > '2020-07-14') & (df4['date'] < '2020-07-28')]

In [1004]:
df4['y4'] = df4['y4_x']

## df5 Workouts 

In [1005]:
dfw = pd.read_csv("14dworkouts.csv")

In [1006]:
llist = []
for index, row in dfw.iterrows():
    x = '2020-01-01' + ' ' + str(row['time_start'])
    llist.append(x)

In [1007]:
dfw['dtstart'] = pd.to_datetime(llist)

In [1008]:
dfw['startstamp'] = (dfw['dtstart'].astype(int) / 10**9)

In [1009]:
col_list = dfw['startstamp'].values.tolist()
scalar = make_interpolater(1577851200, 1577937600, 105, -120) 
col_interp = [scalar(x) for x in col_list] 

In [1010]:
dfw['y5'] = col_interp

In [1011]:
dfw['@workoutActivityType'].value_counts()

HKWorkoutActivityTypeTraditionalStrengthTraining    8
HKWorkoutActivityTypeYoga                           6
Name: @workoutActivityType, dtype: int64

In [1012]:
dfw.rename(columns = {'@workoutActivityType': 'type'}, inplace=True)

In [1013]:
dfw.loc[dfw['type'] == 'HKWorkoutActivityTypeTraditionalStrengthTraining', 'type'] = 'lifting'

In [1014]:
dfw.loc[dfw['type'] == 'HKWorkoutActivityTypeYoga', 'type'] = 'yoga'

In [1015]:
dfw['wcal_chunk'] = dfw['@totalEnergyBurned']/25
dfw['wcal_chunk'] = dfw['wcal_chunk'].round(0).astype(int)

In [1016]:
dfw['@creationDate'] = pd.to_datetime(dfw['@creationDate'])

In [1017]:
dfw['date'] = dfw['@creationDate'].dt.date

In [1018]:
df5 = pd.DataFrame()

In [1019]:
df5 = dfw[['date','type', 'y5', 'wcal_chunk']]

In [1020]:
#df5.head()

## df6 survey data

In [1021]:
dfsd = pd.read_csv('focus_survey_change.csv')

In [1022]:
dfsd.head(2)

,Timestamp,work_type,focus_quality,engagement_rating,company_benefit,comment
0,7/14/20 8:12,admin,4,4,5,prepping forms
1,7/14/20 10:00,creative_generative,4,5,5,personal glyph design


In [1023]:
dfsd.work_type.value_counts()

creative_generative            40
with_people                    14
admin                          14
fucking_off                     8
consuming_learning_strategy     6
consuming_learing_strategy      2
Name: work_type, dtype: int64

In [1024]:
dfsd = dfsd.fillna(0)

In [1025]:
dfsd['datestamp'] = pd.to_datetime(dfsd['Timestamp'])

In [1026]:
dfsd['date'] = dfsd['datestamp'].dt.date

In [1027]:
dfsd['time_s'] = dfsd['datestamp'].dt.time

In [1028]:
dfsd['time_s'] = dfsd['time_s'].astype(str)

In [1029]:
dfsd['t_survey'] = pd.to_datetime(dfsd['time_s'])

In [1030]:
dfsd.loc[(dfsd['t_survey'] > '07:00:00') & (dfsd['t_survey'] < '09:00:00'), 'time_num'] = 8
dfsd.loc[(dfsd['t_survey'] > '09:00:00') & (dfsd['t_survey'] < '11:00:00'), 'time_num'] = 10
dfsd.loc[(dfsd['t_survey'] > '11:00:00') & (dfsd['t_survey'] < '13:00:00'), 'time_num'] = 12
dfsd.loc[(dfsd['t_survey'] > '13:00:00') & (dfsd['t_survey'] < '15:00:00'), 'time_num'] = 14
dfsd.loc[(dfsd['t_survey'] > '15:00:00') & (dfsd['t_survey'] < '17:00:00'), 'time_num'] = 16
dfsd.loc[(dfsd['t_survey'] > '17:00:00') & (dfsd['t_survey'] < '19:00:00'), 'time_num'] = 18

In [1031]:
dfsd['time_num'] = dfsd['time_num'].astype(int)
dfsd['t_num'] = pd.to_datetime(dfsd['time_num'], format = '%H')

In [1032]:
dfsd['t_num_time'] = dfsd['t_num'].dt.time

In [1033]:
klist = []
for index, row in dfsd.iterrows():
    x = '2020-01-01' + ' ' + str(row['t_num_time'])
    klist.append(x)

In [1034]:
dfsd['tmt'] = pd.to_datetime(klist)

In [1035]:
dfsd['tmtstamp'] = (dfsd['tmt'].astype(int) / 10**9)

In [1036]:
col_list = dfsd['tmtstamp'].values.tolist()
scalar = make_interpolater(1577851200, 1577937600, 105, -120) 
col_interp = [scalar(x) for x in col_list] 

In [1037]:
dfsd['y6'] = col_interp

In [1038]:
#dfsurvey.loc[dfsurvey["Are_you_in_isolation"] == 'Yes', 'Are_you_in_isolation'] = 1
#dfsurvey.loc[dfsurvey["Are_you_in_isolation"] == 'No', 'Are_you_in_isolation'] = 2
dfsd.loc[dfsd['focus_quality'] == 1, 'z_focus'] = 25
dfsd.loc[dfsd['focus_quality'] == 2, 'z_focus'] = 150
dfsd.loc[dfsd['focus_quality'] == 3, 'z_focus'] = 275
dfsd.loc[dfsd['focus_quality'] == 4, 'z_focus'] = 425
dfsd.loc[dfsd['focus_quality'] == 5, 'z_focus'] = 575

In [1039]:
dfsd.loc[dfsd['engagement_rating'] == 1, 'z_gage'] = 25
dfsd.loc[dfsd['engagement_rating'] == 2, 'z_gage'] = 150
dfsd.loc[dfsd['engagement_rating'] == 3, 'z_gage'] = 275
dfsd.loc[dfsd['engagement_rating'] == 4, 'z_gage'] = 425
dfsd.loc[dfsd['engagement_rating'] == 5, 'z_gage'] = 575

In [1040]:
dfsd.loc[dfsd['company_benefit'] == 1, 'z_comp'] = 25
dfsd.loc[dfsd['company_benefit'] == 2, 'z_comp'] = 150
dfsd.loc[dfsd['company_benefit'] == 3, 'z_comp'] = 275
dfsd.loc[dfsd['company_benefit'] == 4, 'z_comp'] = 425
dfsd.loc[dfsd['company_benefit'] == 5, 'z_comp'] = 575

In [1041]:
df6 = pd.DataFrame()

In [1042]:
df6 = dfsd[['work_type', 'comment', 'date', 'y6', 'z_focus', 'z_gage', 'z_comp']]

## Start df7  timing nodes

In [1043]:
hrs_list = ['2020-01-01 04:00:00', '2020-01-01 06:00:00', '2020-01-01 08:00:00', '2020-01-01 10:00:00', '2020-01-01 12:00:00', '2020-01-01 14:00:00', '2020-01-01 16:00:00', '2020-01-01 18:00:00', '2020-01-01 20:00:00', '2020-01-01 22:00:00', '2020-01-02 00:00:00', '2020-01-02 02:00:00', '2020-01-02 04:00:00']

In [1044]:
dfhrs = pd.to_datetime(hrs_list)

In [1045]:
dfhrs2 = (dfhrs.astype(int)/10**9)

In [1046]:
# hrs_list = values for every 2 hours from 04 am to next day at 04 am
hrs_list = dfhrs2.values.tolist()

In [1047]:
scalar = make_interpolater(1577851200, 1577937600, -120, 105) 
col_interp = [scalar(x) for x in hrs_list] 
Rcol_interp = Reverse(col_interp)

In [1048]:
df7 = pd.DataFrame()
df7['time_toroid'] = Rcol_interp

In [1049]:
#df7

In [1050]:
# set DEBUG flag
DEBUG = 0

# Initialize all ANTz Node variables
# You shouldn't need to change anything in this section
# you can find an explanation of these terms in Appendix B of the user manual
id = 0
type = 5
data = id
selected = 0
parent_id = 0
branch_level = 0
child_id = id
child_index = 0
palette_id = 0
ch_input_id = 0
ch_output_id = 0
ch_last_updated = 0
average = 0
samples = 0
aux_a_x = 0
aux_a_y = 0
aux_a_z = 0
aux_b_x = 0
aux_b_y = 0
aux_b_z = 0
color_shift = 0
rotate_vec_x = 0
rotate_vec_y = 0
rotate_vec_z = 0
rotate_vec_s = 1
scale_x = 0
scale_y = 0
scale_z = 0
translate_x = 0
translate_y = 0
translate_z = 0
tag_offset_x = 0
tag_offset_y = 0
tag_offset_z = 0
rotate_rate_x = 0
rotate_rate_y = 0
rotate_rate_z = 0
rotate_x = 0
rotate_y = 0
rotate_z = 0
scale_rate_x = 0
scale_rate_y = 0
scale_rate_z = 0
translate_rate_x = 0
translate_rate_y = 0
translate_rate_z = 0
translate_vec_x = 0
translate_vec_y = 0
translate_vec_z = 0
shader = 0
geometry = 7
line_width = 1
point_size = 0
ratio = 0.1
color_index = 0
color_r = 110
color_g = 110
color_b = 110
color_a = 255
color_fade = 0
texture_id = 0
hide = 0
freeze = 0
topo = 3
facet = 0
auto_zoom_x = 0
auto_zoom_y = 0
auto_zoom_z = 0
trigger_hi_x = 0
trigger_hi_y = 0
trigger_hi_z = 0
trigger_lo_x = 0
trigger_lo_y = 0
trigger_lo_z = 1
set_hi_x = 0
set_hi_y = 0
set_hi_z = 0
set_lo_x = 0
set_lo_y = 0
set_lo_z = 0
proximity_x = 0
proximity_y = 0
proximity_z = 0
proximity_mode_x = 0
proximity_mode_y = 0
proximity_mode_z = 0
segments_x = 18
segments_y = 16
segments_z = 0
tag_mode = 0
format_id = 0
table_id = 0
record_id = id
size = 420

In [1051]:
#personal glyph node files
mn = str(initial_value) + "personalglyph_node.csv"

#met_1min tag files
mt = str(initial_value) + "personalglyph_tag.csv"

In [1052]:
# Open ANTz Node file for writing
fout = open(mn,"w")

# Open the Tag file to add metadata to ANTz objects
ftag = open(mt,"w")
# Write the header string
outputstring = "id,record_id,table_id,title,description\n"
ftag.write(outputstring)

# Write rows for header, world, camera views (4), and grid to Node file
outputstring = "id,type,data,selected,parent_id,branch_level,child_id,child_index,palette_id,ch_input_id,ch_output_id,ch_last_updated,average,samples,aux_a_x,aux_a_y,aux_a_z,aux_b_x,aux_b_y,aux_b_z,color_shift,rotate_vec_x,rotate_vec_y,rotate_vec_z,rotate_vec_s,scale_x,scale_y,scale_z,translate_x,translate_y,translate_z,tag_offset_x,tag_offset_y,tag_offset_z,rotate_rate_x,rotate_rate_y,rotate_rate_z,rotate_x,rotate_y,rotate_z,scale_rate_x,scale_rate_y,scale_rate_z,translate_rate_x,translate_rate_y,translate_rate_z,translate_vec_x,translate_vec_y,translate_vec_z,shader,geometry,line_width,point_size,ratio,color_index,color_r,color_g,color_b,color_a,color_fade,texture_id,hide,freeze,topo,facet,auto_zoom_x,auto_zoom_y,auto_zoom_z,trigger_hi_x,trigger_hi_y,trigger_hi_z,trigger_lo_x,trigger_lo_y,trigger_lo_z,set_hi_x,set_hi_y,set_hi_z,set_lo_x,set_lo_y,set_lo_z,proximity_x,proximity_y,proximity_z,proximity_mode_x,proximity_mode_y,proximity_mode_z,segments_x,segments_y,segments_z,tag_mode,format_id,table_id,record_id,size\n"
fout.write(outputstring)

# Row for world parameters
outputstring = "1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Row for first camera view
outputstring = "2,1,2,0,0,0,2,2,3,0,0,0,0,1,0,0,0,0,0,0,0,0,0.008645,0.825266,-0.564678,1,1,1,-32.446629,-180.908295,143.514175,0,0,1,0,0,0,55.620094,0.600200,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,214.306686,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Row for second camera view
outputstring = "3,1,3,0,2,1,3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,1,1,1,-0.500000,0,571.750000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Third camera view
outputstring = "4,1,4,0,2,1,4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,-0,1,1,1,0,-90,7,0,0,1,0,0,0,90,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Fourth camera view
outputstring = "5,1,5,0,2,1,5,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,-0,-0,1,1,1,85,0,7,0,0,1,0,0,0,90,270,0,0,0,0,0,0,0,-0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Default Grid
outputstring = "6,6,6,1,0,0,0,1,0,0,0,0,0,1,700,250,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,3,0,0,255,150,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,420\n"
fout.write(outputstring)

# Initialize increments
taginc = 0
id = 6

numlevel0toroids = 1
numlevel1toroids = 1

for index, row in df1.iterrows() :
    id += 1
    data = id
    type = 5
    parent_id = 0
    selected = 0
    branch_level = 0
    child_id = id
    scale_x = 1
    scale_y = scale_x
    scale_z = scale_x 
    translate_x = row['xroot']
    translate_y = row['yroot']
    translate_z = row['zroot']
    rotate_x = 0
    rotate_y = 0
    rotate_z = 0
    ratio = .1
    line_width= 1
    geometry = 16
    topo = 5
    color_r = 24
    color_g = 123
    color_b = 205
    color_a = 255
    record_id = id 
    texture_id= 0

    # Creat output string and write to ANTz Node File
    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
    fout.write(outputstring)

    # Output to Tag file
    tagtext = "Date: " + str(row['date'])
    tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
    ftag.write(tagstring)
    taginc += 1

    level0objectid = id
    # grid object
    for j in range(numlevel1toroids):    
        id += 1
        data = id
        type = 5
        parent_id = level0objectid
        selected = 0
        branch_level = 1
        child_id = id
        scale_x = .27
        scale_y = scale_x
        scale_z = scale_x 
        translate_x = 0
        translate_y = 0
        translate_z = 0
        rotate_x = 0
        rotate_y = 0
        rotate_z = 0
        ratio = .1
        line_width= 1
        geometry = 20
        topo = 8
        color_r = 24
        color_g = 123
        color_b = 205
        color_a = 50
        record_id = id 
        segments_x = 8
        segments_y = 10
        aux_a_x = 30
        aux_a_y = 30
        aux_a_z = 0

        # Creat output string and write to ANTz Node File
        outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
        fout.write(outputstring)

        # Output to Tag file
        tagtext = ""
        tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
        ftag.write(tagstring)
        taginc += 1

        level1objectid = id
        
        for i7, row7 in df7.iterrows():
            id += 1
            data = id
            type = 5
            parent_id = level1objectid
            selected = 0
            branch_level = 2
            child_id = id
            translate_x = -120
            translate_y = row7['time_toroid']
            translate_z = 1
            scale_x = 3.5 
            scale_y = scale_x
            scale_z = scale_x
            ratio = .1
            line_width= 1
            geometry = 11
            topo = 3
            rotate_x = 0
            rotate_y = 0
            color_r = 163
            color_g = 192
            color_b = 223
            color_a = 175
            record_id = id 
            texture_id= 0

            # Creat output string and write to ANTz Node File
            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
            fout.write(outputstring)
        
        numlevel2toroids = 3
        #k are top 3 rods
        for k in range(numlevel2toroids):
            if k == 0:
                id += 1
                data = id
                type = 5
                parent_id = level1objectid
                selected = 0
                branch_level = 2
                child_id = id
                translate_x = -120
                translate_y = 146
                translate_z = 3
                scale_x = 7 
                scale_y = scale_x
                scale_z = 8
                ratio = .3
                line_width= 1
                geometry = 19
                topo = 6
                rotate_x = 90
                rotate_y = 90
                color_r = 158
                color_g = 168
                color_b = 186
                color_a = 255
                record_id = id 
                texture_id= 0

                # Creat output string and write to ANTz Node File
                outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                fout.write(outputstring)

                level2objectid = id

                # sleep score data

                numlevel3toroids = 8

                for m in range(numlevel3toroids):
                    id += 1
                    data = id
                    type = 5
                    parent_id = level2objectid
                    selected = 0
                    branch_level = 3
                    child_id = id
                    if m == 0:
                        translate_x = 45
                        color_r = 102
                        color_g = 0
                        color_b = 51
                        ratio = row['score_scale'] 
                        scale_x = 3.8
                        scale_y = scale_x
                        scale_z = scale_x
                        geometry = 3
                        topo = 2
                        tagtext = "Sleep Score: " + str(row['score'])
                    elif m == 1:
                        translate_x = 0
                        color_r = 153
                        color_g = 0
                        color_b = 76
                        ratio = row['score_alignment_scale'] 
                        scale_x = 3.8
                        scale_y = scale_x
                        scale_z = scale_x
                        geometry = 7
                        topo = 3
                        tagtext = "Sleep score - alignment: " + str(row['score_alignment'])
                    elif m == 2: 
                        translate_x = -30
                        color_r = 204
                        color_g = 0
                        color_b = 102
                        ratio = row['score_deep_scale']
                        scale_x = 3.8
                        scale_y = scale_x
                        scale_z = scale_x
                        geometry = 7
                        topo = 3
                        tagtext = "Sleep score - deep: " + str(row['score_deep'])
                    elif m == 3:
                        translate_x = -60
                        color_r = 255
                        color_g = 0
                        color_b = 127
                        ratio = row['score_rem_scale']
                        scale_x = 3.8
                        scale_y = scale_x
                        scale_z = scale_x
                        geometry = 7
                        topo = 3
                        tagtext = "Sleep score - rem: " + str(row['score_rem'])
                    elif m == 4:
                        translate_x = -90
                        color_r = 255
                        color_g = 51
                        color_b = 153
                        ratio = row['score_disturbances_scale']
                        scale_x = 3.8
                        scale_y = scale_x
                        scale_z = scale_x
                        geometry = 7
                        topo = 3
                        tagtext = "Sleep score - disturbances: " + str(row['score_disturbances'])
                    elif m == 5:
                        translate_x = -120
                        color_r = 255
                        color_g = 102
                        color_b = 178
                        ratio = row['score_total_scale']
                        scale_x = 3.8
                        scale_y = scale_x
                        scale_z = scale_x
                        geometry = 7
                        topo = 3
                        tagtext = "Sleep score - total time: " + str(row['score_total'])
                    elif m == 6:
                        translate_x = -150
                        color_r = 255
                        color_g = 153
                        color_b = 204
                        ratio = row['score_efficiency_scale']
                        scale_x = 3.8
                        scale_y = scale_x
                        scale_z = scale_x
                        geometry = 7
                        topo = 3
                        tagtext = "Sleep score - efficiency: " + str(row['score_efficiency'])
                    elif m == 7: 
                        translate_x = -180
                        color_r = 255
                        color_g = 204
                        color_b = 229
                        ratio = row['score_latency_scale']
                        scale_x = 3.8
                        scale_y = scale_x
                        scale_z = scale_x
                        geometry = 7
                        topo = 3
                        tagtext = "Sleep score - latency: " + str(row['score_latency'])
                    translate_y = 0
                    translate_z = 0
                    line_width= 1
                    color_a = 255
                    rotate_x = 0
                    rotate_y = 0
                    record_id = id 
                    texture_id= 0

                    # Creat output string and write to ANTz Node File
                    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                    fout.write(outputstring)

                    # Output to Tag file 
                    tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                    ftag.write(tagstring)
                    taginc += 1

            elif k == 1: 
                id += 1
                data = id
                type = 5
                parent_id = level1objectid
                selected = 0
                branch_level = 2
                child_id = id
                translate_x = -120
                translate_y = -146
                translate_z = 3
                scale_x = 7 
                scale_y = scale_x
                scale_z = 8
                ratio = .3
                line_width= 1
                geometry = 19
                topo = 6
                rotate_x = 90
                rotate_y = 90
                color_r = 158
                color_g = 168
                color_b = 186
                color_a = 255
                record_id = id 
                texture_id= 0

                # Creat output string and write to ANTz Node File
                outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                fout.write(outputstring)

                level2objectid = id
                #activity score data
                numlevel3toroids = 7
                for n in range(numlevel3toroids):
                    id += 1
                    data = id
                    type = 5
                    parent_id = level2objectid
                    selected = 0
                    branch_level = 3
                    child_id = id
                    if n == 0:
                        translate_x = 45
                        color_r = 51
                        color_g = 0
                        color_b = 102
                        ratio = row['score_x_scale'] 
                        scale_x = 3.8
                        scale_y = scale_x
                        scale_z = scale_x
                        geometry = 3
                        topo = 2
                        tagtext = "Activity Score: " + str(row['score_x'])
                    elif n == 1:
                        translate_x = 0
                        color_r = 76
                        color_g = 0
                        color_b = 153
                        ratio = row['score_meet_daily_targets_scale'] 
                        scale_x = 3.8
                        scale_y = scale_x
                        scale_z = scale_x
                        geometry = 7
                        topo = 3
                        tagtext = "Activity score - hit daily targets: " + str(row['score_meet_daily_targets'])
                    elif n == 2: 
                        translate_x = -36
                        color_r = 102
                        color_g = 0
                        color_b = 204
                        ratio = row['score_move_every_hour_scale']
                        scale_x = 3.8
                        scale_y = scale_x
                        scale_z = scale_x
                        geometry = 7
                        topo = 3
                        tagtext = "Activity score - move each hour: " + str(row['score_move_every_hour'])
                    elif n == 3:
                        translate_x = -72
                        color_r = 127
                        color_g = 0
                        color_b = 255
                        ratio = row['score_recovery_time_scale']
                        scale_x = 3.8
                        scale_y = scale_x
                        scale_z = scale_x
                        geometry = 7
                        topo = 3
                        tagtext = "Activity score - recovery time: " + str(row['score_recovery_time'])
                    elif n == 4:
                        translate_x = -108
                        color_r = 153
                        color_g = 51
                        color_b = 255
                        ratio = row['score_stay_active_scale']
                        scale_x = 3.8
                        scale_y = scale_x
                        scale_z = scale_x
                        geometry = 7
                        topo = 3
                        tagtext = "Activity score - stay active: " + str(row['score_stay_active'])
                    elif n == 5:
                        translate_x = -144
                        color_r = 178
                        color_g = 102
                        color_b = 255
                        ratio = row['score_training_frequency_scale']
                        scale_x = 3.8
                        scale_y = scale_x
                        scale_z = scale_x
                        tagtext = "Activity score - training frequency: " + str(row['score_training_frequency'])
                        geometry = 7
                        topo = 3
                    elif n == 6:
                        translate_x = -180
                        color_r = 204
                        color_g = 153
                        color_b = 255
                        ratio = row['score_training_volume_scale']
                        scale_x = 3.8
                        scale_y = scale_x
                        scale_z = scale_x
                        tagtext = "Activity score - training volume: " + str(row['score_training_volume'])
                        geometry = 7
                        topo = 3
                    translate_y = 0
                    translate_z = 0
                    rotate_x = 0
                    rotate_y = 0
                    line_width= 1
                    color_a = 255
                    record_id = id 
                    texture_id= 0

                    # Creat output string and write to ANTz Node File
                    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                    fout.write(outputstring)

                    # Output to Tag file 
                    tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                    ftag.write(tagstring)
                    taginc += 1


            elif k == 2: 
                id += 1
                data = id
                type = 5
                parent_id = level1objectid
                selected = 0
                branch_level = 2
                child_id = id
                translate_x = -120
                translate_y = 125
                translate_z = 3
                scale_x = 7 
                scale_y = scale_x
                scale_z = 8
                ratio = .3
                line_width= 1
                geometry = 19
                topo = 6
                rotate_x = 90
                rotate_y = 90
                color_r = 158
                color_g = 168
                color_b = 186
                color_a = 255
                record_id = id 
                texture_id= 0

                # Creat output string and write to ANTz Node File
                outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                fout.write(outputstring)

                level2objectid = id
                numlevel3toroids = 9
                # readiness score data
                for o in range(numlevel3toroids):
                    id += 1
                    data = id
                    type = 5
                    parent_id = level2objectid
                    selected = 0
                    branch_level = 3
                    child_id = id
                    if o == 0:
                        translate_x = 45
                        color_r = 0
                        color_g = 51
                        color_b = 102
                        ratio = row['score_y_scale'] 
                        scale_x = 3.8
                        scale_y = scale_x
                        scale_z = scale_x
                        geometry = 3
                        topo = 2
                        tagtext = "Readiness Score: " + str(row['score_y'])
                    elif o == 1:
                        translate_x = 0
                        color_r = 0
                        color_g = 76
                        color_b = 153
                        ratio = row['score_activity_balance_scale'] 
                        scale_x = 3.8
                        scale_y = scale_x
                        scale_z = scale_x
                        geometry = 7
                        topo = 3
                        tagtext = "Readiness score - activity balance: " + str(row['score_activity_balance'])
                    elif o == 2: 
                        translate_x = -26
                        color_r = 0
                        color_g = 102
                        color_b = 204
                        ratio = row['score_hrv_balance_scale']
                        scale_x = 3.8
                        scale_y = scale_x
                        scale_z = scale_x
                        geometry = 7
                        topo = 3
                        tagtext = "Readiness score - hrv balance: " + str(row['score_hrv_balance'])
                    elif o == 3:
                        translate_x = -52
                        color_r = 0
                        color_g = 128
                        color_b = 255
                        ratio = row['score_previous_day_scale']
                        scale_x = 3.8
                        scale_y = scale_x
                        scale_z = scale_x
                        geometry = 7
                        topo = 3
                        tagtext = "Readiness score - previous day: " + str(row['score_previous_day'])
                    elif o == 4:
                        translate_x = -78
                        color_r = 51
                        color_g = 153
                        color_b = 255
                        ratio = row['score_previous_night_scale']
                        scale_x = 3.8
                        scale_y = scale_x
                        scale_z = scale_x
                        geometry = 7
                        topo = 3
                        tagtext = "Readiness score - previous night: " + str(row['score_previous_night'])
                    elif o == 5:
                        translate_x = -104
                        color_r = 102
                        color_g = 178
                        color_b = 255
                        ratio = row['score_recovery_index_scale']
                        scale_x = 3.8
                        scale_y = scale_x
                        scale_z = scale_x
                        geometry = 7
                        topo = 3
                        tagtext = "Readiness score - recovery index: " + str(row['score_recovery_index'])
                    elif o == 6:
                        translate_x = -130
                        color_r = 153
                        color_g = 204
                        color_b = 255
                        ratio = row['score_resting_hr_scale']
                        scale_x = 3.8
                        scale_y = scale_x
                        scale_z = scale_x
                        geometry = 7
                        topo = 3
                        tagtext = "Readiness score - resting hr: " + str(row['score_resting_hr'])
                    elif o == 7: 
                        translate_x = -156
                        color_r = 153
                        color_g = 204
                        color_b = 255
                        ratio = row['score_sleep_balance_scale']
                        scale_x = 3.8
                        scale_y = scale_x
                        scale_z = scale_x
                        geometry = 7
                        topo = 3
                        tagtext = "Readiness score - sleep balance: " + str(row['score_sleep_balance'])
                    elif o == 8: 
                        translate_x = -180
                        color_r = 204
                        color_g = 229
                        color_b = 255
                        ratio = row['score_temperature_scale']
                        scale_x = 3.8
                        scale_y = scale_x
                        scale_z = scale_x
                        geometry = 7
                        topo = 3
                        tagtext = "Readiness score - temperature: " + str(row['score_temperature'])
                    translate_y = 0
                    translate_z = 0
                    line_width= 1
                    rotate_x = 0
                    rotate_y = 0
                    color_a = 255
                    record_id = id 
                    texture_id= 0

                    # Creat output string and write to ANTz Node File
                    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                    fout.write(outputstring)

                    # Output to Tag file 
                    tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                    ftag.write(tagstring)
                    taginc += 1
        
        #bb are food meal glyphs
        for i2, row2 in df3.loc[df3['date'] == row['date']].iterrows():
            id += 1
            data = id
            type = 5
            parent_id = level1objectid
            selected = 0
            branch_level = 2
            child_id = id
            translate_x = -90
            translate_y = row2['y3']
            translate_z = 3
            scale_x = 5.3 
            scale_y = scale_x
            scale_z = scale_x
            ratio = .18
            line_width= 1
            geometry = 7
            topo = 3
            color_r = 99
            color_g = 101
            color_b = 120
            color_a = 175
            record_id = id 
            texture_id= 0

            # Creat output string and write to ANTz Node File
            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
            fout.write(outputstring)

            level2objectid = id 
            numlevel3toroids = 5
            for dd in range(numlevel3toroids):
                id += 1
                data = id
                type = 5
                parent_id = level2objectid
                selected = 0
                branch_level = 3
                child_id = id
                if dd == 0:
                    translate_x = 90
                    translate_y = 0
                    translate_z = 0
                    scale_x = row2['p_scale'] 
                    scale_y = scale_x
                    scale_z = scale_x
                    geometry = 19
                    topo = 4
                    color_r = 194
                    color_g = 33
                    color_b = 33
                if dd == 1:
                    translate_x = 155
                    translate_y = 0
                    translate_z = 0
                    scale_x = row2['s_scale'] 
                    scale_y = scale_x
                    scale_z = scale_x
                    geometry = 1
                    topo = 1
                    color_r = 255
                    color_g = 183
                    color_b = 51
                if dd == 2:
                    translate_x = -180
                    translate_y = 0
                    translate_z = 0
                    scale_x = row2['c_scale'] 
                    scale_y = scale_x
                    scale_z = scale_x
                    geometry = 19
                    topo = 4
                    color_r = 245
                    color_g = 115
                    color_b = 54
                if dd == 3:
                    translate_x = -155
                    translate_y = 0
                    translate_z = 0
                    scale_x = row2['fb_scale'] 
                    scale_y = scale_x
                    scale_z = scale_x
                    geometry = 3
                    topo = 2
                    color_r = 127
                    color_g = 22
                    color_b = 55
                if dd == 4:
                    translate_x = -90
                    translate_y = 0
                    translate_z = 0
                    scale_x = row2['ft_scale'] 
                    scale_y = scale_x
                    scale_z = scale_x
                    geometry = 19
                    topo = 4
                    color_r = 4
                    color_g = 120
                    color_b = 120
                ratio = .1
                line_width= 1
                color_a = 255
                record_id = id 
                texture_id= 0

                # Creat output string and write to ANTz Node File
                outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                fout.write(outputstring)

            numlevel3toroids = row2['cal_chunk']
            for ee in range(numlevel3toroids):
                id += 1
                data = id
                type = 5
                parent_id = level2objectid
                selected = 0
                branch_level = 3
                child_id = id
                translate_x = ((70/numlevel3toroids)*ee)+(-35)
                translate_y = 0
                translate_z = 0
                scale_x = 1 
                scale_y = scale_x
                scale_z = scale_x
                geometry = 16
                topo = 5
                color_r = 109
                color_g = 19
                color_b = 194
                ratio = .1
                line_width= 1
                color_a = 255
                record_id = id 
                texture_id= 0

                # Creat output string and write to ANTz Node File
                outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                fout.write(outputstring)
                                                    
        #ff are blood glucose glyphs
        for i4, row4 in df4.loc[df4['date'] == row['date']].iterrows():
            id += 1
            data = id
            type = 5
            parent_id = level1objectid
            selected = 0
            branch_level = 2
            child_id = id
            translate_x = -30
            translate_y = row4['y4']
            translate_z = 3
            scale_x = 2 
            scale_y = scale_x
            scale_z = scale_x
            ratio = .11
            line_width= 1
            rotate_x = 90
            rotate_y = 270
            geometry = 19
            topo = 6
            color_r = 115
            color_g = 108
            color_b = 20
            color_a = 170
            record_id = id 
            texture_id= 0

            # Creat output string and write to ANTz Node File
            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
            fout.write(outputstring)

            level2objectid = id 
            numlevel3toroids = 3
            for gg in range(numlevel3toroids):
                id += 1
                data = id
                type = 5
                parent_id = level2objectid
                selected = 0
                branch_level = 3
                child_id = id
                if gg == 0:
                    translate_x = 0
                    translate_y = 0
                    translate_z = 0
                    color_r = 242
                    color_g = 162
                    color_b = 92
                    ratio = row4['bg_scale']
                if gg == 1:
                    translate_x = -90
                    translate_y = 0
                    translate_z = 0
                    color_r = 148
                    color_g = 225
                    color_b = 242
                    ratio = row4['k_scale']
                if gg == 2:
                    translate_x = -180
                    translate_y = 0
                    translate_z = 0
                    color_r = 217
                    color_g = 137
                    color_b = 137
                    ratio = row4['gki_scale']
                scale_x = 3.8 
                scale_y = scale_x
                scale_z = scale_x
                geometry = 7
                topo = 3
                rotate_y = 90
                line_width= 1
                color_a = 255
                record_id = id 
                texture_id= 0

                # Creat output string and write to ANTz Node File
                outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                fout.write(outputstring) 
                    
                    
        #kk are workout glyphs
        for i5, row5 in df5.loc[df5['date'] == row['date']].iterrows():
            id += 1
            data = id
            type = 5
            parent_id = level1objectid
            selected = 0
            branch_level = 2
            child_id = id
            translate_x = 30
            translate_y = row5['y5']
            translate_z = 3
            scale_x = 4.5 
            scale_y = scale_x
            scale_z = scale_x
            ratio = .1
            line_width= 1
            rotate_x = 0
            rotate_y = 0
            if row5['type'] == 'yoga':
                geometry = 3
                color_r = 180
                color_g = 196
                color_b = 217
            elif row5['type'] == 'lifting':
                geometry = 15
                color_r = 4
                color_g = 157
                color_b = 191
            topo = 2
            color_a = 255
            record_id = id 
            texture_id= 0

            # Creat output string and write to ANTz Node File
            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
            fout.write(outputstring)

            level2objectid = id 
            numlevel3toroids = row5['wcal_chunk']
            for ii in range(numlevel3toroids):
                id += 1
                data = id
                type = 5
                parent_id = level2objectid
                selected = 0
                branch_level = 3
                child_id = id
                translate_x = ((190/numlevel3toroids)*ii)+(-95)
                translate_y = ((20/numlevel3toroids)*ii)+(20)
                translate_z = 60
                color_r = 242
                color_g = 75
                color_b = 75
                ratio = .1
                scale_x = 1 
                scale_y = scale_x
                scale_z = scale_x
                geometry = 16
                topo = 5
                line_width= 1
                color_a = 255
                record_id = id 
                texture_id= 0

                # Creat output string and write to ANTz Node File
                outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                fout.write(outputstring)
                            
        #nn are survey glyphs
        for i6, row6 in df6.loc[df6['date'] == row['date']].iterrows():
            id += 1
            data = id
            type = 5
            parent_id = level1objectid
            selected = 0
            branch_level = 2
            child_id = id
            translate_x = 90
            translate_y = row6['y6']
            translate_z = 3
            scale_x = 26 
            scale_y = 8
            scale_z = 8
            ratio = .1
            line_width= 1
            rotate_x = 0
            rotate_y = 0
            geometry = 1
            topo = 1
            if row6['work_type'] == 'admin':
                color_r = 217
                color_g = 201
                color_b = 173
            elif row6['work_type'] == 'creative_generative':
                color_r = 242
                color_g = 92
                color_b = 132
            elif row6['work_type'] == 'with_people':
                color_r = 4
                color_g = 191
                color_b = 157
            elif row6['work_type'] == 'fucking_off':
                color_r = 242
                color_g = 159
                color_b = 5
            elif row6['work_type'] == 'consuming_learning_strategy':
                color_r = 191
                color_g = 73
                color_b = 150
            elif row6['work_type'] == 'consuming_learing_strategy':
                color_r = 191
                color_g = 73
                color_b = 150
            color_a = 255
            record_id = id 
            texture_id= 0
            
            # Creat output string and write to ANTz Node File
            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
            fout.write(outputstring)

            level2objectid = id 
            numlevel3toroids = 3
            for ss in range(numlevel3toroids):
                id += 1
                data = id
                type = 5
                parent_id = level2objectid
                selected = 0
                branch_level = 3
                child_id = id
                if ss == 0:
                    translate_x = -140
                    translate_y = -7
                    translate_z = row6['z_gage']
                    color_r = 245
                    color_g = 213
                    color_b = 60
                if ss == 1:
                    translate_x = 0
                    translate_y = -6
                    translate_z = row6['z_focus']
                    color_r = 166
                    color_g = 46
                    color_b = 137
                if ss == 2:
                    translate_x = 131
                    translate_y = 20
                    translate_z = row6['z_comp']
                    color_r = 55
                    color_g = 166
                    color_b = 164
                ratio = .1
                scale_x = .44 
                scale_y = 1.3
                scale_z = .36
                geometry = 16
                topo = 5
                line_width= 1
                color_a = 255
                record_id = id
                facet = 5
                texture_id= 0

                # Creat output string and write to ANTz Node File
                outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                fout.write(outputstring)
                    
        numlevel2toroids = 1
        # bedtime start glyph
        for k in range(numlevel2toroids):
            id += 1
            data = id
            type = 5
            parent_id = level1objectid
            selected = 0
            branch_level = 2
            child_id = id
            translate_x = -115
            translate_y = row['bedtime_start']
            translate_z = 3
            scale_x = 5.5 
            scale_y = scale_x
            scale_z = scale_x
            ratio = .1
            line_width= 1
            geometry = 9
            topo = 2
            color_r = 82
            color_g = 16
            color_b = 168
            color_a = 255
            record_id = id 
            facet = 0
            texture_id= 0

            # Creat output string and write to ANTz Node File
            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
            fout.write(outputstring)